In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dir = '/content/drive/My Drive/'

# Dataset & DataLoader

In [6]:
import json
import glob
import os

import numpy as np
import cv2
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import functional as F

In [8]:
class RoadDataset(Dataset):
    def __init__(self, args, split, img_transform, label_transform):
        self.args = args
        self.data_dir = dir
        self.split = split
        self.targets = []

        if self.split == "valid":
            self.sample_dir = os.path.join(self.data_dir, "train")
        else:
            self.sample_dir = os.path.join(self.data_dir, split)

        self.input_files = glob.glob(os.path.join(self.sample_dir, "samples/*png"))
        self.target_files = glob.glob(os.path.join(self.sample_dir, "targets/*json"))

        for f in self.target_files:
            img = os.path.join(self.sample_dir, "samples", os.path.splitext(os.path.basename(f))[0] + ".png")
            assert os.path.isfile(img)
            self.targets.append([f, img])
        self.targets.sort()

        if self.split == 'train':
            self.input_files = self.input_files[:-int(len(self.input_files) * self.args.valid_ratio)]
            self.targets = self.targets[:-int(len(self.targets) * self.args.valid_ratio)]

        elif self.split == 'valid':
            self.input_files = self.input_files[-int(len(self.input_files) * self.args.valid_ratio):]
            self.targets = self.targets[-int(len(self.targets) * self.args.valid_ratio):]

        self.img_transform = img_transform
        self.label_transform = label_transform
    
        self.road_colorbook = {"Mortorway":(51, 51, 255),
                               "Primary":(51, 255, 255), 
                               "Secondary":(51, 255, 51), 
                               "Tertiary":(255, 255, 51), 
                               "Residential":(255, 51, 51), 
                               "Unclassified":(255, 51, 255), 
                               "background":(0, 0, 0)}

        self.road_class_id = {"Mortorway":1, 
                              "Primary":2, 
                              "Secondary":3, 
                              "Tertiary":4, 
                              "Residential":5, 
                              "Unclassified":6, 
                              "background":0}

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        X = self.input_files[idx]
        with open(self.targets[idx][0], "r") as jfile:
            meta = json.load(jfile)
        y = self.make_mask((1024, 1024), meta)
        img = Image.open(X)
        # mask = Image.open(y)

        if self.img_transform:
            image = self.img_transform(img)
            mask = self.label_transform(y)
        
        return image, mask
        
    def make_mask(self, size, label):
        mask = np.zeros([size[0], size[1]], dtype=np.uint8)
        for r in range(len(label["features"])):
            road = label["features"][r]["properties"]
            type_name = road["type_name"]
            temp = road["road_imcoords"].split(",")
            coords = np.array([int(round(float(c))) for c in temp]).reshape(-1, 2)
            cv2.fillPoly(mask, [coords], self.road_class_id[type_name])
        return mask


In [70]:
# import argparse
# parser = argparse.ArgumentParser(description='PyTorch Semantic Segmentation Training')
# args = parser.parse_args('')
# args.valid_ratio = 0.5

# img_trans = transforms.Compose([transforms.Resize((512, 512)),
#                                 transforms.ToTensor()])
# label_trans = transforms.Compose([transforms.ToPILImage(),
#                                   transforms.Resize((512, 512)),
#                                   transforms.PILToTensor()])

# trainset = RoadDataset(args=args, split='train', img_transform=img_trans, label_transform=label_trans)
# train_loader = DataLoader(trainset, batch_size=2)

# samples, targets = next(iter(train_loader))
# print(samples.shape, targets.squeeze(1).shape)
# print(np.unique(targets))

torch.Size([2, 3, 512, 512]) torch.Size([2, 512, 512])
[0 1 2 3 4 5 6]


In [9]:
def make_data_loaders(args):

    img_trans = transforms.Compose([transforms.Resize((512, 512)),
                                    transforms.ToTensor()])
    label_trans = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Resize((512, 512)),
                                      transforms.PILToTensor()])

    trainset = RoadDataset(args=args, split='train', img_transform=img_trans, label_transform=label_trans)
    validset = RoadDataset(args=args, split='valid', img_transform=img_trans, label_transform=label_trans)
    testset = RoadDataset(args=args, split='test', img_transform=img_trans, label_transform=label_trans)

    train_loader = DataLoader(trainset,
                              batch_size=args.batch_size,
                              pin_memory=True)

    valid_loader = DataLoader(validset,
                              batch_size=args.batch_size,
                              pin_memory=True)

    test_loader = DataLoader(testset,
                             batch_size=args.batch_size,
                             pin_memory=True)

    return train_loader, valid_loader, test_loader

In [74]:
# import argparse
# parser = argparse.ArgumentParser(description='PyTorch Semantic Segmentation Training')
# args = parser.parse_args('')
# args.valid_ratio = 0.5
# args.batch_size = 2

# train_loader, valid_loader, test_loader = make_data_loaders(args)

# samples, targets = next(iter(train_loader))
# print(samples.shape, targets.squeeze(1).shape)
# samples, targets = next(iter(valid_loader))
# print(samples.shape, targets.squeeze(1).shape)
# samples, targets = next(iter(test_loader))
# print(samples.shape, targets.squeeze(1).shape)

torch.Size([2, 3, 512, 512]) torch.Size([2, 512, 512])
torch.Size([2, 3, 512, 512]) torch.Size([2, 512, 512])
torch.Size([2, 3, 512, 512]) torch.Size([2, 512, 512])


# Model

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
def load_segmentation_models(model_name, num_class):

    if model_name == 'fcn8':
        return FCN8s(n_class=num_class)
    elif model_name == 'fcn16':
        return FCN16s(n_class=num_class)
    elif model_name == 'fcn32':
        return FCN32s(n_class=num_class)
    elif model_name == 'unet':
        return unet(num_classes=num_class)
    elif model_name == 'segnet':
        return SegNet(num_classes=num_class)
    
def get_upsampling_weight(in_channels, out_channels, kernel_size):
    """Make a 2D bilinear kernel suitable for upsampling"""
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size),
                      dtype=np.float64)
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight).float()

In [12]:
class FCN32s(nn.Module):

    def __init__(self, n_class=6):
        super(FCN32s, self).__init__()
        # conv1
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/2

        # conv2
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/4

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/8

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/16

        # conv5
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()

        self.score_fr = nn.Conv2d(4096, n_class, 1)
        self.upscore = nn.ConvTranspose2d(n_class, n_class, 64, stride=32,
                                          bias=False)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.zero_()
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                assert m.kernel_size[0] == m.kernel_size[1]
                initial_weight = get_upsampling_weight(
                    m.in_channels, m.out_channels, m.kernel_size[0])
                m.weight.data.copy_(initial_weight)

    def forward(self, x):
        h = x
        h = self.relu1_1(self.conv1_1(h))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        h = self.pool3(h)

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        h = self.pool4(h)

        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)

        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        h = self.score_fr(h)

        h = self.upscore(h)
        h = h[:, :, 19:19 + x.size()[2], 19:19 + x.size()[3]].contiguous()

        return h

In [13]:
class FCN16s(nn.Module):

    def __init__(self, n_class=6):
        super(FCN16s, self).__init__()
        # conv1
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/2

        # conv2
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/4

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/8

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/16

        # conv5
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()

        self.score_fr = nn.Conv2d(4096, n_class, 1)
        self.score_pool4 = nn.Conv2d(512, n_class, 1)

        self.upscore2 = nn.ConvTranspose2d(
            n_class, n_class, 4, stride=2, bias=False)
        self.upscore16 = nn.ConvTranspose2d(
            n_class, n_class, 32, stride=16, bias=False)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.zero_()
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                assert m.kernel_size[0] == m.kernel_size[1]
                initial_weight = get_upsampling_weight(
                    m.in_channels, m.out_channels, m.kernel_size[0])
                m.weight.data.copy_(initial_weight)

    def forward(self, x):
        h = x
        h = self.relu1_1(self.conv1_1(h))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        h = self.pool3(h)

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        h = self.pool4(h)
        pool4 = h  # 1/16

        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)

        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        h = self.score_fr(h)
        h = self.upscore2(h)
        upscore2 = h  # 1/16

        h = self.score_pool4(pool4)
        h = h[:, :, 5:5 + upscore2.size()[2], 5:5 + upscore2.size()[3]]
        score_pool4c = h  # 1/16

        h = upscore2 + score_pool4c

        h = self.upscore16(h)
        h = h[:, :, 27:27 + x.size()[2], 27:27 + x.size()[3]].contiguous()

        return h

In [14]:
class FCN8s(nn.Module):

    def __init__(self, n_class=6):
        super(FCN8s, self).__init__()
        # conv1
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/2

        # conv2
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/4

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/8

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/16

        # conv5
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()

        self.score_fr = nn.Conv2d(4096, n_class, 1)
        self.score_pool3 = nn.Conv2d(256, n_class, 1)
        self.score_pool4 = nn.Conv2d(512, n_class, 1)

        self.upscore2 = nn.ConvTranspose2d(
            n_class, n_class, 4, stride=2, bias=False)
        self.upscore8 = nn.ConvTranspose2d(
            n_class, n_class, 16, stride=8, bias=False)
        self.upscore_pool4 = nn.ConvTranspose2d(
            n_class, n_class, 4, stride=2, bias=False)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.zero_()
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                assert m.kernel_size[0] == m.kernel_size[1]
                initial_weight = get_upsampling_weight(
                    m.in_channels, m.out_channels, m.kernel_size[0])
                m.weight.data.copy_(initial_weight)

    def forward(self, x):
        h = x
        h = self.relu1_1(self.conv1_1(h))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        h = self.pool3(h)
        pool3 = h  # 1/8

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        h = self.pool4(h)
        pool4 = h  # 1/16

        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)

        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        h = self.score_fr(h)
        h = self.upscore2(h)
        upscore2 = h  # 1/16

        h = self.score_pool4(pool4)
        h = h[:, :, 5:5 + upscore2.size()[2], 5:5 + upscore2.size()[3]]
        score_pool4c = h  # 1/16

        h = upscore2 + score_pool4c  # 1/16
        h = self.upscore_pool4(h)
        upscore_pool4 = h  # 1/8

        h = self.score_pool3(pool3)
        h = h[:, :, 9:9 + upscore_pool4.size()[2], 9:9 + upscore_pool4.size()[3]]
        score_pool3c = h  # 1/8

        h = upscore_pool4 + score_pool3c  # 1/8

        h = self.upscore8(h)
        h = h[:, :, 31:31 + x.size()[2], 31:31 + x.size()[3]].contiguous()

        return h

In [81]:
# model = load_segmentation_models('fcn8', 6)
# for image, mask in train_loader:
#     y = model(image)
#     print(y.shape)
#     break

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([2, 6, 512, 512])


In [15]:
class CBR(nn.Module):

    def __init__(self, c_in, c_out, k_size, stride, padding):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=k_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=c_out)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)

        return x

class CR(nn.Module):

    def __init__(self, c_in, c_out, k_size, stride, padding):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=k_size, stride=stride, padding=padding)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.act(x)

        return x

class UNet(nn.Module):

    def __init__(self, num_classes):
        super().__init__()

        # Contracting path
        self.encoder_1_1 = CR(3, 64, k_size=3, stride=1, padding=1)
        self.encoder_1_2 = CR(64, 64, k_size=3, stride=1, padding=1)
        self.encoder_1_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_2_1 = CR(64, 128, k_size=3, stride=1, padding=1)
        self.encoder_2_2 = CR(128, 128, k_size=3, stride=1, padding=1)
        self.encoder_2_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_3_1 = CR(128, 256, k_size=3, stride=1, padding=1)
        self.encoder_3_2 = CR(256, 256, k_size=3, stride=1, padding=1)
        self.encoder_3_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_4_1 = CR(256, 512, k_size=3, stride=1, padding=1)
        self.encoder_4_2 = CR(512, 512, k_size=3, stride=1, padding=1)
        self.encoder_4_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_5_1 = CR(512, 1024, k_size=3, stride=1, padding=1)
        self.encoder_5_2 = CR(1024, 1024, k_size=3, stride=1, padding=1)

        # Expansive path
        self.decoder_1_1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2, padding=0)
        self.decoder_1_2 = CR(1024, 512, k_size=3, stride=1, padding=1)
        self.decoder_1_3 = CR(512, 512, k_size=3, stride=1, padding=1)

        self.decoder_2_1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0)
        self.decoder_2_2 = CR(512, 256, k_size=3, stride=1, padding=1)
        self.decoder_2_3 = CR(256, 256, k_size=3, stride=1, padding=1)

        self.decoder_3_1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=0)
        self.decoder_3_2 = CR(256, 128, k_size=3, stride=1, padding=1)
        self.decoder_3_3 = CR(128, 128, k_size=3, stride=1, padding=1)

        self.decoder_4_1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2, padding=0)
        self.decoder_4_2 = CR(128, 64, k_size=3, stride=1, padding=1)
        self.decoder_4_3 = CR(64, 64, k_size=3, stride=1, padding=1)

        self.pointwise_conv = nn.Conv2d(64, num_classes, kernel_size=1, stride=1, padding=0)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            if isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):

        # Contracting path
        enc_1_1 = self.encoder_1_1(x)
        enc_1_2 = self.encoder_1_2(enc_1_1)
        pool_1 = self.encoder_1_3(enc_1_2)

        enc_2_1 = self.encoder_2_1(pool_1)
        enc_2_2 = self.encoder_2_2(enc_2_1)
        pool_2 = self.encoder_2_3(enc_2_2)

        enc_3_1 = self.encoder_3_1(pool_2)
        enc_3_2 = self.encoder_3_2(enc_3_1)
        pool_3 = self.encoder_3_3(enc_3_2)

        enc_4_1 = self.encoder_4_1(pool_3)
        enc_4_2 = self.encoder_4_2(enc_4_1)
        pool_4 = self.encoder_4_3(enc_4_2)

        enc_5_1 = self.encoder_5_1(pool_4)
        enc_5_2 = self.encoder_5_2(enc_5_1)

        # Expansive path
        up_conv_1 = self.decoder_1_1(enc_5_2, output_size=enc_4_1.shape[2:])
        dec_1_2 = self.decoder_1_2(torch.cat([enc_4_2, up_conv_1], dim=1))
        dec_1_3 = self.decoder_1_3(dec_1_2)

        up_conv_2 = self.decoder_2_1(dec_1_3, output_size=enc_3_1.shape[2:])
        dec_2_2 = self.decoder_2_2(torch.cat([enc_3_2, up_conv_2], dim=1))
        dec_2_3 = self.decoder_2_3(dec_2_2)

        up_conv_3 = self.decoder_3_1(dec_2_3, output_size=enc_2_1.shape[2:])
        dec_3_2 = self.decoder_3_2(torch.cat([enc_2_2, up_conv_3], dim=1))
        dec_3_3 = self.decoder_3_3(dec_3_2)

        up_conv_4 = self.decoder_4_1(dec_3_3, output_size=enc_1_1.shape[2:])
        dec_4_2 = self.decoder_4_2(torch.cat([enc_1_2, up_conv_4], dim=1))
        dec_4_3 = self.decoder_4_3(dec_4_2)
        
        out = self.pointwise_conv(dec_4_3)

        return out

class UNet_BN(nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        
        # Contracting path
        self.encoder_1_1 = CBR(3, 64, k_size=3, stride=1, padding=1)
        self.encoder_1_2 = CBR(64, 64, k_size=3, stride=1, padding=1)
        self.encoder_1_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_2_1 = CBR(64, 128, k_size=3, stride=1, padding=1)
        self.encoder_2_2 = CBR(128, 128, k_size=3, stride=1, padding=1)
        self.encoder_2_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_3_1 = CBR(128, 256, k_size=3, stride=1, padding=1)
        self.encoder_3_2 = CBR(256, 256, k_size=3, stride=1, padding=1)
        self.encoder_3_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_4_1 = CBR(256, 512, k_size=3, stride=1, padding=1)
        self.encoder_4_2 = CBR(512, 512, k_size=3, stride=1, padding=1)
        self.encoder_4_3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.encoder_5_1 = CBR(512, 1024, k_size=3, stride=1, padding=1)
        self.encoder_5_2 = CBR(1024, 1024, k_size=3, stride=1, padding=1)

        # Expansive path
        self.decoder_1_1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2, padding=0)
        self.decoder_1_2 = CBR(1024, 512, k_size=3, stride=1, padding=1)
        self.decoder_1_3 = CBR(512, 512, k_size=3, stride=1, padding=1)

        self.decoder_2_1 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0)
        self.decoder_2_2 = CBR(512, 256, k_size=3, stride=1, padding=1)
        self.decoder_2_3 = CBR(256, 256, k_size=3, stride=1, padding=1)

        self.decoder_3_1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=0)
        self.decoder_3_2 = CBR(256, 128, k_size=3, stride=1, padding=1)
        self.decoder_3_3 = CBR(128, 128, k_size=3, stride=1, padding=1)

        self.decoder_4_1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2, padding=0)
        self.decoder_4_2 = CBR(128, 64, k_size=3, stride=1, padding=1)
        self.decoder_4_3 = CBR(64, 64, k_size=3, stride=1, padding=1)

        self.pointwise_conv = nn.Conv2d(64, num_classes, kernel_size=1, stride=1, padding=0)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            if isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):

        # Contracting path
        enc_1_1 = self.encoder_1_1(x)
        enc_1_2 = self.encoder_1_2(enc_1_1)
        pool_1 = self.encoder_1_3(enc_1_2)

        enc_2_1 = self.encoder_2_1(pool_1)
        enc_2_2 = self.encoder_2_2(enc_2_1)
        pool_2 = self.encoder_2_3(enc_2_2)

        enc_3_1 = self.encoder_3_1(pool_2)
        enc_3_2 = self.encoder_3_2(enc_3_1)
        pool_3 = self.encoder_3_3(enc_3_2)

        enc_4_1 = self.encoder_4_1(pool_3)
        enc_4_2 = self.encoder_4_2(enc_4_1)
        pool_4 = self.encoder_4_3(enc_4_2)

        enc_5_1 = self.encoder_5_1(pool_4)
        enc_5_2 = self.encoder_5_2(enc_5_1)

        # Expansive path
        up_conv_1 = self.decoder_1_1(enc_5_2, output_size=enc_4_1.shape[2:])
        dec_1_2 = self.decoder_1_2(torch.cat([enc_4_2, up_conv_1], dim=1))
        dec_1_3 = self.decoder_1_3(dec_1_2)

        up_conv_2 = self.decoder_2_1(dec_1_3, output_size=enc_3_1.shape[2:])
        dec_2_2 = self.decoder_2_2(torch.cat([enc_3_2, up_conv_2], dim=1))
        dec_2_3 = self.decoder_2_3(dec_2_2)

        up_conv_3 = self.decoder_3_1(dec_2_3, output_size=enc_2_1.shape[2:])
        dec_3_2 = self.decoder_3_2(torch.cat([enc_2_2, up_conv_3], dim=1))
        dec_3_3 = self.decoder_3_3(dec_3_2)

        up_conv_4 = self.decoder_4_1(dec_3_3, output_size=enc_1_1.shape[2:])
        dec_4_2 = self.decoder_4_2(torch.cat([enc_1_2, up_conv_4], dim=1))
        dec_4_3 = self.decoder_4_3(dec_4_2)
        
        out = self.pointwise_conv(dec_4_3)

        return out


def unet(num_classes, use_batchnorm=False):
    if use_batchnorm:
        return UNet_BN(num_classes)
    else:
        return UNet(num_classes)

In [16]:
class SegNet(nn.Module):
    """SegNet: A Deep Convolutional Encoder-Decoder Architecture for
    Image Segmentation. https://arxiv.org/abs/1511.00561
    See https://github.com/alexgkendall/SegNet-Tutorial for original models.
    Args:
        num_classes (int): number of classes to segment
        n_init_features (int): number of input features in the fist convolution
        drop_rate (float): dropout rate of each encoder/decoder module
        filter_config (list of 5 ints): number of output features at each level
    """
    def __init__(self, num_classes, n_init_features=3, drop_rate=0.5,
                 filter_config=(64, 128, 256, 512, 512)):
        super(SegNet, self).__init__()

        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        # setup number of conv-bn-relu blocks per module and number of filters
        encoder_n_layers = (2, 2, 3, 3, 3)
        encoder_filter_config = (n_init_features,) + filter_config
        decoder_n_layers = (3, 3, 3, 2, 1)
        decoder_filter_config = filter_config[::-1] + (filter_config[0],)

        for i in range(0, 5):
            # encoder architecture
            self.encoders.append(_Encoder(encoder_filter_config[i],
                                          encoder_filter_config[i + 1],
                                          encoder_n_layers[i], drop_rate))

            # decoder architecture
            self.decoders.append(_Decoder(decoder_filter_config[i],
                                          decoder_filter_config[i + 1],
                                          decoder_n_layers[i], drop_rate))

        # final classifier (equivalent to a fully connected layer)
        self.classifier = nn.Conv2d(filter_config[0], num_classes, 3, 1, 1)

    def forward(self, x):
        indices = []
        unpool_sizes = []
        feat = x

        # encoder path, keep track of pooling indices and features size
        for i in range(0, 5):
            (feat, ind), size = self.encoders[i](feat)
            indices.append(ind)
            unpool_sizes.append(size)

        # decoder path, upsampling with corresponding indices and size
        for i in range(0, 5):
            feat = self.decoders[i](feat, indices[4 - i], unpool_sizes[4 - i])

        return self.classifier(feat)


class _Encoder(nn.Module):
    def __init__(self, n_in_feat, n_out_feat, n_blocks=2, drop_rate=0.5):
        """Encoder layer follows VGG rules + keeps pooling indices
        Args:
            n_in_feat (int): number of input features
            n_out_feat (int): number of output features
            n_blocks (int): number of conv-batch-relu block inside the encoder
            drop_rate (float): dropout rate to use
        """
        super(_Encoder, self).__init__()

        layers = [nn.Conv2d(n_in_feat, n_out_feat, 3, 1, 1),
                  nn.BatchNorm2d(n_out_feat),
                  nn.ReLU(inplace=True)]

        if n_blocks > 1:
            layers += [nn.Conv2d(n_out_feat, n_out_feat, 3, 1, 1),
                       nn.BatchNorm2d(n_out_feat),
                       nn.ReLU(inplace=True)]
            if n_blocks == 3:
                layers += [nn.Dropout(drop_rate)]

        self.features = nn.Sequential(*layers)

    def forward(self, x):
        output = self.features(x)
        return F.max_pool2d(output, 2, 2, return_indices=True), output.size()


class _Decoder(nn.Module):
    """Decoder layer decodes the features by unpooling with respect to
    the pooling indices of the corresponding decoder part.
    Args:
        n_in_feat (int): number of input features
        n_out_feat (int): number of output features
        n_blocks (int): number of conv-batch-relu block inside the decoder
        drop_rate (float): dropout rate to use
    """
    def __init__(self, n_in_feat, n_out_feat, n_blocks=2, drop_rate=0.5):
        super(_Decoder, self).__init__()

        layers = [nn.Conv2d(n_in_feat, n_in_feat, 3, 1, 1),
                  nn.BatchNorm2d(n_in_feat),
                  nn.ReLU(inplace=True)]

        if n_blocks > 1:
            layers += [nn.Conv2d(n_in_feat, n_out_feat, 3, 1, 1),
                       nn.BatchNorm2d(n_out_feat),
                       nn.ReLU(inplace=True)]
            if n_blocks == 3:
                layers += [nn.Dropout(drop_rate)]

        self.features = nn.Sequential(*layers)

    def forward(self, x, indices, size):
        unpooled = F.max_unpool2d(x, indices, 2, 2, 0, size)
        return self.features(unpooled)

# Metric

In [17]:
class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

    def _generate_matrix(self, gt_image, pred_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pred_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        Acc = np.nanmean(Acc)
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
               np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0)
               - np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)

        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU

    def add_batch(self, gt_image, pred_image):
        assert gt_image.shape == pred_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pred_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

# Arguments

In [18]:
import torch.optim as optim
from tqdm import tqdm
import random
import os
import pandas as pd
import argparse

import warnings
warnings.filterwarnings("ignore")

In [19]:
def segmentation_parser():
    parser = argparse.ArgumentParser(description='PyTorch Semantic Segmentation Training')

    # data loader
    parser.add_argument('--valid_ratio', type=float, default=0.5, help='validation set ratio')
    parser.add_argument('--shuffle_dataset', type=bool, default=True)

    # model type selection
    parser.add_argument('--n_class', type=int, default=7, help='# of class')
    parser.add_argument('--model_name', type=str, default="segnet",
                        choices=["fcn8", "fcn16", "fcn32", "unet", "segnet"],
                        help='segmentation models')

    # training hyperparameters
    parser.add_argument('--epochs', type=int, default=5,
                        help='number of epochs to train (default: auto)')
    parser.add_argument('--batch-size', type=int, default=2,
                        metavar='N', help='input batch size for training (default: auto)')
    parser.add_argument('--test-batch-size', type=int, default=1,
                        metavar='N', help='input batch size for training (default: auto)')

    # optimizer parameters
    parser.add_argument('--lr', type=float, default=1e-3,
                        help='learning rate (default: auto)')
    parser.add_argument('--weight-decay', type=float, default=1e-2,
                        metavar='M', help='weight decay (default: 5e-4)')
    parser.add_argument('--momentum', type=float, default=0.9)

    # cuda, seed and logging
    parser.add_argument('--seed', type=int, default=2021,
                        help='random seed (default: 1)')
    parser.add_argument('--results_dir', type=str, default='./results')

    return parser

In [20]:
# define parser
parser = segmentation_parser()
args = parser.parse_args('')
args.cuda = torch.device('cuda:0')
args.model_name = "fcn8"
print(args)
# reproducibility
random_seed = args.seed
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

Namespace(batch_size=2, cuda=device(type='cuda', index=0), epochs=5, lr=0.001, model_name='fcn8', momentum=0.9, n_class=7, results_dir='./results', seed=2021, shuffle_dataset=True, test_batch_size=1, valid_ratio=0.5, weight_decay=0.01)


# Experiment

In [21]:
# define data loader
train_loader, valid_loader, test_loader = make_data_loaders(args)

# define model
model = load_segmentation_models(model_name=args.model_name, num_class=args.n_class)
model.to(args.cuda)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train(net, data_loader, train_optimizer, epoch, args):
    net.train()

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader, desc="train steps", total=len(data_loader))
    for inputs, targets in train_bar:
        targets = targets.squeeze(1).long()
        inputs, targets = inputs.to(args.cuda), targets.to(args.cuda)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, get_lr(train_optimizer), total_loss / total_num))

    return total_loss / total_num


def validation(net, data_loader, evaluator, epoch, args):
    net.eval()
    evaluator.reset()

    total_loss, total_num, valid_bar = 0.0, 0, tqdm(data_loader, desc="validation steps", total=len(data_loader))
    for inputs, targets in valid_bar:
        targets = targets.squeeze(1).long()
        inputs, targets = inputs.to(args.cuda), targets.to(args.cuda)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        targets = targets.cpu().numpy()
        _, predict = torch.max(outputs, dim=1)
        pred = predict.cpu().numpy()

        evaluator.add_batch(targets, pred)

        Acc = evaluator.Pixel_Accuracy()
        # Acc_class = evaluator.Pixel_Accuracy_Class()
        mIoU = evaluator.Mean_Intersection_over_Union()
        # FWIoU = evaluator.Frequency_Weighted_Intersection_over_Union()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        valid_bar.set_description('Valid Epoch: [{}/{}], Loss: {:.4f}, Acc: {:.3f}, mIoU: {:.3f}'.format(epoch,
                                                                                                         args.epochs,
                                                                                                         total_loss / total_num,
                                                                                                         Acc,
                                                                                                         mIoU))

    return total_loss / total_num, Acc, mIoU


def test(net, data_loader, evaluator, epoch, args):
    net.eval()
    evaluator.reset()

    total_loss, total_num, test_bar = 0.0, 0, tqdm(data_loader, desc="test steps", total=len(data_loader))
    for inputs, targets in test_bar:
        targets = targets.squeeze(1).long()
        inputs, targets = inputs.to(args.cuda), targets.to(args.cuda)

        outputs = model(inputs)

        targets = targets.cpu().numpy()
        _, predict = torch.max(outputs, dim=1)
        pred = predict.cpu().numpy()

        evaluator.add_batch(targets, pred)

        Acc = evaluator.Pixel_Accuracy()
        # Acc_class = evaluator.Pixel_Accuracy_Class()
        mIoU = evaluator.Mean_Intersection_over_Union()
        # FWIoU = evaluator.Frequency_Weighted_Intersection_over_Union()

        total_num += data_loader.batch_size
        test_bar.set_description('Test Epoch: [{}/{}], Acc: {:.3f}, mIoU: {:.3f}'.format(epoch,
                                                                                          args.epochs,
                                                                                          Acc,
                                                                                          mIoU))

    return Acc, mIoU

In [22]:
#%% training setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
evaluator = Evaluator(num_class=args.n_class)
best_pred = -np.inf

#%% logging
results = {'train_loss': [], 'valid_loss':[], 'valid_acc':[], 'valid_mIoU':[], 'test_acc':[], 'test_mIoU':[]}

if not os.path.exists(args.results_dir):
    os.makedirs(args.results_dir, exist_ok=True)

#%% training run !
epoch_start = 1

for epoch in range(epoch_start, args.epochs+1):
    # train
    train_loss = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)

    valid_loss, valid_acc, valid_mIoU = validation(model, valid_loader, evaluator, epoch, args)
    results['valid_loss'].append(valid_loss)
    results['valid_acc'].append(valid_acc)
    results['valid_mIoU'].append(valid_mIoU)

    if valid_mIoU > best_pred:
        test_acc, test_mIoU = test(model, test_loader, evaluator, epoch, args)

        torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()},
                   args.results_dir + '/model_last.pth')

        best_pred = valid_mIoU

    results['test_acc'].append(test_acc)
    results['test_mIoU'].append(test_mIoU)

    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log.csv', index_label='epoch')

Train Epoch: [1/5], lr: 0.001000, Loss: 1.9459: 100%|██████████| 1/1 [00:03<00:00,  3.66s/it]
Valid Epoch: [1/5], Loss: 1.9421, Acc: 0.774, mIoU: 0.111: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it]
Train Epoch: [2/5], lr: 0.001000, Loss: 1.9417: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]
Valid Epoch: [2/5], Loss: 1.9381, Acc: 0.774, mIoU: 0.111: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]
Train Epoch: [3/5], lr: 0.001000, Loss: 1.9372: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]
Valid Epoch: [3/5], Loss: 1.9336, Acc: 0.774, mIoU: 0.111: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]
Train Epoch: [4/5], lr: 0.001000, Loss: 1.9323: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]
Valid Epoch: [4/5], Loss: 1.9287, Acc: 0.774, mIoU: 0.111: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]
Train Epoch: [5/5], lr: 0.001000, Loss: 1.9268: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]
Valid Epoch: [5/5], Loss: 1.9234, Acc: 0.774, mIoU: 0.111: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]


In [29]:
data = pd.read_csv('./results/log.csv')
print(data)

   epoch  train_loss  valid_loss  valid_acc  valid_mIoU  test_acc  test_mIoU
0      1    1.945908    1.942139   0.774485    0.110641  0.795927   0.113704
1      2    1.941730    1.938060   0.774485    0.110641  0.795927   0.113704
2      3    1.937205    1.933599   0.774485    0.110641  0.795927   0.113704
3      4    1.932259    1.928710   0.774485    0.110641  0.795927   0.113704
4      5    1.926837    1.923370   0.774485    0.110641  0.795927   0.113704
